In [1]:
from buttermilk import BM
import os
from hydra import initialize, compose
from omegaconf import OmegaConf


from promptflow.tracing import trace, start_trace
from rich import print as rprint

os.chdir('../..')
print(os.getcwd())

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="./conf"):
    cfg = compose(config_name='config',
                  overrides=["+flows=summarise_osb","job=export",])



# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)




/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/suzor/src/buttermilk
2024-11-12 17:22:21 J5HW6L4KT6 buttermilk buttermilk.py[ 232] INFO {'message': "Logging setup for: 20241112T0722Z-nrs4-J5HW6L4KT6-suzor. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'automod', 'job': 'export', 'task_id': '20241112T0722Z-nrs4-J5HW6L4KT6-suzor'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/automod/export/20241112T0722Z-nrs4-J5HW6L4KT6-suzor", 'project': 'automod', 'job': 'export', 'run_id': '20241112T0722Z-nrs4-J5HW6L4KT6-suzor', 'save_dir': 'gs://dmrc-analysis/runs/automod/export/20241112T0722Z-nrs4-J5HW6L4KT6-suzor', 'ip': '159.196.210.27', 'node_name': 'J5HW6L4KT6', 'username': 'suzor'}
2024-11-12 17:22:21 J5HW6L4KT6 buttermilk buttermilk.py[ 240] DEBUG Buttermilk version is: 0.2.0
Prompt flow service has started...


{
    'name': 'automod',
    'job': 'export',
    'logger': {'type': 'gcp'},
    'tracing': {'enabled': True},
    'verbose': True,
    'save_dest': {'type': 'gcp', 'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'},
    'secret_provider': {
        'type': 'azure',
        'vault': 'https://suzorvault.vault.azure.net/',
        'resource_group': 'rg-suzor_ai',
        'models_secret': 'models'
    },
    'run': {'platform': 'local', 'max_concurrency': 32},
    'flows': [
        {
            'name': 'summarise OSB cases',
            'num_runs': 3,
            'concurrency': 20,
            'agent': {
                'type': 'LC',
                'template': 'summarise',
                'instructions': 'summarise_osb',
                'model': ['gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'record': 'record'},
            'data': [
                {
                    'name': 'osb all cases',
                    'type': 'file',
                    'uri': 'gs://dmrc-platforms/data/osb/cases.json',
                    'columns': {'record_id': 'record_id', 'text': 'text'}
                }
            ],
            'save': {
                'type': 'bq',
                'dataset': 'dmrc-analysis.toxicity.flow',
                'schema': 'buttermilk/schemas/flow.json'
            }
        },
        {
            'name': 'synthesise OSB cases',
            'num_runs': 1,
            'concurrency': 20,
            'agent': {
                'type': 'LC',
                'template': 'synthesise',
                'instructions': 'summarise_osb',
                'model': ['gpt4o', 'sonnet']
            },
            'parameters': {'content': 'text', 'answers': 'answers'},
            'data': [
                {
                    'name': 'osb all cases',
                    'type': 'file',
                    'uri': 'gs://dmrc-platforms/data/osb/cases.json',
                    'columns': {'record_id': 'record_id', 'text': 'text'}
                },
                {
                    'name': 'synthesise previous runs',
                    'type': 'job',
                    'dataset': 'dmrc-analysis.toxicity.flow',
                    'max_records_per_group': 4,
                    'last_n_days': 1,
                    'agg': True,
                    'filter': {
                        'agent_info.flow': 'summarise OSB cases',
                        'parameters.model': ['gpt4o', 'sonnet', 'gemini15pro']
                    },
                    'join': {'record_id': 'record.record_id'},
                    'group': {
                        'flow': 'agent_info.flow',
                        'template': 'agent_info.template',
                        'model': 'parameters.model'
                    },
                    'columns': {
                        'answers': {'id': 'job_id', 'text': 'outputs'},
                        'meta': {
                            'flow': 'agent_info.flow',
                            'template': 'agent_info.template',
                            'agent_info': 'agent_info',
                            'model': 'parameters.model',
                            'timestamp': 'timestamp'
                        }
                    }
                }
            ],
            'save': {
                'type': 'bq',
                'dataset': 'dmrc-analysis.toxicity.flow',
                'schema': 'buttermilk/schemas/flow.json'
            }
        },
        {
            'name': 'export',
            'agent': {'type': 'GSheetExporter', 'sheet_name': 'OSB summaries'},
            'save': {'type': 'gsheets', 'destination': 'new'},
            'data': [
                {
                    'name': 'osb all cases',
                    'type': 'file',
                    'uri': 'gs://dmrc-platforms/data/osb/cases.json',
                    'columns': {'record_id': 'record_id', 'text': 'text'}
                },
                {
        

I0000 00:00:1731396141.828363 64159249 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


2024-11-12 17:22:22 J5HW6L4KT6 buttermilk save.py[ 272] DEBUG Uploading file gs://dmrc-analysis/runs/automod/export/20241112T0722Z-nrs4-J5HW6L4KT6-suzor/config_7AcHxZ.json.


DEBUG:buttermilk:Uploading file gs://dmrc-analysis/runs/automod/export/20241112T0722Z-nrs4-J5HW6L4KT6-suzor/config_7AcHxZ.json.


2024-11-12 17:22:23 J5HW6L4KT6 buttermilk save.py[ 275] DEBUG Successfully uploaded file gs://dmrc-analysis/runs/automod/export/20241112T0722Z-nrs4-J5HW6L4KT6-suzor/config_7AcHxZ.json with 6216 lines written.


DEBUG:buttermilk:Successfully uploaded file gs://dmrc-analysis/runs/automod/export/20241112T0722Z-nrs4-J5HW6L4KT6-suzor/config_7AcHxZ.json with 6216 lines written.


In [ ]:
from buttermilk.runner.helpers import load_data
from buttermilk.utils.gsheet import GSheet

from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(flow=cfg.flows[2], source="notebook")

await orchestrator.prepare()

df = orchestrator._dataset
df


2024-11-12 17:22:31 J5HW6L4KT6 buttermilk buttermilk.py[ 312] INFO Query stats: Ran in 0:00:02.684585 seconds, cache hit: None, billed None, approx cost $unknown.


INFO:buttermilk:Query stats: Ran in 0:00:02.684585 seconds, cache hit: None, billed None, approx cost $unknown.


,record_id,text,model,timestamp,title,date,content,reasons,recommendations,result,type,topics,standards,location,outputs
74,BUN-QBBLZ8WI,"{""result"": NULL, ""format"": ""bundle"", ""title"": ...",gpt4o,2024-11-12 02:06:58.143588+00:00,Mention of Al-Shabaab,2023-11-22,Two Facebook posts referred to the terrorist g...,[Tier 1 of the Dangerous Individuals and Organ...,[Meta should assess how well reviewers underst...,leave up,summary,"[War and conflict, Dangerous individuals and o...",[Dangerous Individuals and Organizations policy],Somalia,{'content': 'Two Facebook posts referred to th...
66,IG-H3138H6S,"{""result"": ""upheld"", ""title"": ""Violence agains...",gpt4o,2024-11-12 02:07:00.228930+00:00,Violence against women,2023-03-09,Two Instagram posts condemned gender-based vio...,[Tier 1 of the Hate Speech Community Standard ...,[Meta should explicitly include an exception i...,leave up,standard,"[Freedom of expression, Sex and gender equalit...","[Hate speech, Hate Speech Community Standard, ...",Sweden,{'content': 'Two Instagram posts condemned gen...
48,IG-I9DP23IB,"{""result"": ""upheld"", ""title"": ""Öcalan's isolat...",sonnet,2024-11-12 02:07:32.479087+00:00,Öcalan's isolation,2021-06-29,An Instagram user in the United States posted ...,[The Dangerous Individuals and Organizations p...,[Immediately restore the misplaced 2017 guidan...,leave up,standard,"[Freedom of expression, Marginalized communiti...",[Dangerous Individuals and Organizations],"Turkey, United States",{'content': 'An Instagram user in the United S...
51,FB-T8JDDDJV,"{""result"": ""upheld"", ""title"": ""Political dispu...",sonnet,2024-11-12 05:52:18.199051+00:00,Political dispute ahead of Turkish elections,2023-02-08,Three Turkish media organizations posted video...,"[The term ""İngiliz uşağı"" does not constitute ...",[Revise the Hate Speech Community Standard to ...,leave up,standard,"[Elections, Journalism, Natural disasters, Pol...","[Hate Speech Community Standard, Newsworthines...",Turkey,{'content': 'Three Turkish media organizations...
75,FB-4294T386,"{""result"": ""upheld"", ""title"": ""Fruit juice die...",gpt4o,2024-11-12 02:07:16.861893+00:00,Fruit juice diet,2023-05-01,Two videos posted on Facebook featured intervi...,[The Suicide and Self-Injury Community Standar...,[Meta should restrict extreme and harmful diet...,leave up,standard,"[Children's rights, Health]",[Suicide and Self-Injury Community Standard],Italy,{'content': 'Two videos posted on Facebook fea...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,IG-WUC3649N,"{""result"": ""upheld"", ""title"": ""Al-Shifa Hospit...",gpt4o,2024-11-12 02:07:11.767293+00:00,Al-Shifa Hospital,2023-12-19,A video showing the aftermath of a strike on o...,[The Violent and Graphic Content policy prohib...,[Amend the Violent and Graphic Content Communi...,interstitial,expedited,"[Safety, Violence, War and conflict, Humanitar...",[Violent and Graphic Content Community Standard],Gaza,{'content': 'A video showing the aftermath of ...
16,FB-QBJDASCV,"{""result"": ""upheld"", ""title"": ""Armenians in Az...",gpt4o,2024-11-12 05:51:31.403080+00:00,Armenians in Azerbaijan,2020-11,A user shared a post containing historical pho...,[The Hate Speech Community Standard prohibits ...,[The platform should provide users with clear ...,takedown,standard,"[Culture, Discrimination, Religion, Armed Conf...",[Hate Speech Community Standard],"Armenia, Azerbaijan",{'content': 'A user shared a post containing h...
19,FB-L1LANIA7,"{""result"": ""upheld"", ""title"": ""Wampum belt"",""a...",gpt4o,2024-11-12 02:07:09.621025+00:00,Wampum belt,2021-08-27,An Indigenous North American artist posted an ...,[Tier 1 of the Hate Speech Community Standard ...,[Provide users with timely and accurate notifi...,leave up,standard,"[Art, Indigenous rights, Cultural expression, ...",[Hate Speech Community Standard],"Canada, United States",{'content': 'An Indigenous North American arti...
44,IG-2R3UEQRR,"{""result"": ""upheld

In [3]:
# save the input data from this step to a spreadsheet so that we can compare later.
from buttermilk.utils.gsheet import GSheet, format_strings

answers = format_strings(orchestrator._dataset, convert_json_columns=['outputs'])

gsheet = GSheet()
sheet = gsheet.save_gsheet(df=answers,  sheet_name="syntheses", title="OSB Summaries")
sheet_id = sheet.id



2024-11-12 17:22:51 J5HW6L4KT6 buttermilk gsheet.py[ 118] INFO Saved to https://docs.google.com/spreadsheets/d/12HMeQ3yYrvh-bmEtK-DSEEQNZtQEwwkOz4HtX8-e8Y8


INFO:buttermilk:Saved to https://docs.google.com/spreadsheets/d/12HMeQ3yYrvh-bmEtK-DSEEQNZtQEwwkOz4HtX8-e8Y8


In [13]:
# rotate the output around and save it to a new sheet

import pandas as pd
import json
pivot = orchestrator._dataset.pivot(index=["record_id","timestamp"],columns=["model"], values=["content","reasons","recommendations"])
pivot.columns = pivot.columns.droplevel()

_ = gsheet.save_gsheet(df=pivot, sheet_id=sheet_id, sheet_name="pivot", title="OSB Summaries")

# evals = prepare_step_data(cfg.data, cfg.step[3].data)
# evals = evals.set_index(["record_id","model"]).sort_values("timestamp", ascending=False).reset_index().drop_duplicates(["record_id", "model"])
# evals = format_strings(evals, convert_json_columns=['analysis'])
# evals = evals.set_index(["record_id"]).pivot(columns="model",values="analysis")
# sheet = gsheet.save_gsheet(df=evals, sheet_id=sheet_id, sheet_name="evals", title="results")



2024-11-12 17:45:56 J5HW6L4KT6 buttermilk gsheet.py[ 118] INFO Saved to https://docs.google.com/spreadsheets/d/12HMeQ3yYrvh-bmEtK-DSEEQNZtQEwwkOz4HtX8-e8Y8


INFO:buttermilk:Saved to https://docs.google.com/spreadsheets/d/12HMeQ3yYrvh-bmEtK-DSEEQNZtQEwwkOz4HtX8-e8Y8


Index(['gpt4o', 'sonnet', 'gpt4o', 'sonnet', 'gpt4o', 'sonnet'], dtype='object', name='model')